In [ ]:
%%sql -r dataframe_7
USE SCHEMA AI_DEMOS.CALL_CENTER_DEMO;

In [ ]:
# pretty markdown printing
import json
from rich.markdown import Markdown
from rich.console import Console
console = Console()

from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
from snowflake.cortex import complete
session = get_active_session()
root = Root(session)

# Transcribe and Analyze Call Recordings

In [ ]:
%%sql -r dataframe_1
SELECT
  RELATIVE_PATH,
  SIZE,
  FILE_URL
FROM
  DIRECTORY('@CALL_CENTER_RECORDINGS');

In [ ]:
%%sql -r dataframe_2
CREATE TABLE IF NOT EXISTS CALL_CENTER_TRANSCRIPTIONS AS 
SELECT
  RELATIVE_PATH,
  
  -- Call Transcribe-Function with speaker-diarization
  AI_TRANSCRIBE(TO_FILE('@CALL_CENTER_RECORDINGS', RELATIVE_PATH),{'timestamp_granularity': 'speaker'}) AS TRANSCRIPTION,
  f.value['speaker_label']::TEXT AS SPEAKER,
  f.value['text']::TEXT AS CALL_TRANSCRIPTION,
  f.value['start']::FLOAT AS SEGMENT_START,
  f.value['end']::FLOAT AS SEGMENT_END,
  
  -- Call Sentiment Analysis for transcriptions
  AI_SENTIMENT(CALL_TRANSCRIPTION)['categories'][0]['sentiment']::TEXT AS TRANSCRIPTION_SENTIMENT,
  CASE TRANSCRIPTION_SENTIMENT
    WHEN 'positive' THEN 1
    WHEN 'negative' THEN -1
        -- Group 'neutral', 'mixed', and 'unknown' together as 0
    WHEN 'neutral' THEN 0
    WHEN 'mixed' THEN 0
    WHEN 'unknown' THEN 0
    -- Optional: Handle any unexpected values safely
    ELSE 0 
  END AS SENTIMENT
FROM 
  DIRECTORY('@CALL_CENTER_RECORDINGS'),
  LATERAL FLATTEN(INPUT => TRANSCRIPTION['segments']) f;


-- View Transcriptions
SELECT
  RELATIVE_PATH,
  SPEAKER,
  CALL_TRANSCRIPTION,
  TRANSCRIPTION_SENTIMENT,
  SENTIMENT,
  SEGMENT_START,
  SEGMENT_END
FROM
  CALL_CENTER_TRANSCRIPTIONS
WHERE
  SPEAKER = 'SPEAKER_00'
ORDER BY
  RELATIVE_PATH,
  SEGMENT_START;

In [ ]:
%%sql -r dataframe_3
-- View aggregated Transcriptions
SELECT
  RELATIVE_PATH,
  AVG(SENTIMENT) AS AVERAGE_SENTIMENT
FROM
  CALL_CENTER_TRANSCRIPTIONS
WHERE
  SPEAKER = 'SPEAKER_00'
GROUP BY RELATIVE_PATH;

# Extract Documents & Make them Searchable

In [ ]:
%%sql -r dataframe_4
SELECT
  RELATIVE_PATH,
  SIZE,
  FILE_URL
FROM
  DIRECTORY('@DOCUMENTS');

In [ ]:
%%sql -r dataframe_5
CREATE TABLE IF NOT EXISTS CALL_CENTER_DOCUMENTS AS 
SELECT
  RELATIVE_PATH, 
  -- Extract Document
  AI_PARSE_DOCUMENT(TO_FILE('@DOCUMENTS', RELATIVE_PATH), {'mode': 'LAYOUT'})['content']::TEXT AS DOCUMENT_CONTENT
FROM
  DIRECTORY('@DOCUMENTS');

-- View Extractions
SELECT
  *
FROM 
  CALL_CENTER_DOCUMENTS;

In [ ]:
%%sql -r dataframe_6
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS call_center_docs
  ON DOCUMENT_CONTENT
  ATTRIBUTES RELATIVE_PATH
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 hour'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
    RELATIVE_PATH,
    DOCUMENT_CONTENT
  FROM
    CALL_CENTER_DOCUMENTS
);

# Retrieval Augmented Generation

### Query Cortex Search & Generate Responses using LLMs

In [ ]:
# Connect to Cortex Search Service
search_service = (root
  .databases["AI_DEMOS"]
  .schemas["CALL_CENTER_DEMO"]
  .cortex_search_services["call_center_docs"]
)

In [ ]:
# [R] 
# Retrieve relevant documents
question = "What is the expected Average Speed of Answer for customer Europe Tech Solutions AG?"

resp = search_service.search(
  query=question,
  columns=["RELATIVE_PATH", "DOCUMENT_CONTENT"],
  limit=1
)


# [A] 
# Augment Context with retrieved document
context = resp.results[0]['DOCUMENT_CONTENT']

llm_prompt = f"""
Anwser this question based on the provided context.
Question: {question}
Context: {context}
"""


# [G] 
# Generate Response
llm_response = complete('claude-sonnet-4-6', llm_prompt)
console.print(Markdown(llm_response))

## Retrieved Document

In [ ]:
console.print(Markdown(json.loads(resp.to_json())['results'][0]['DOCUMENT_CONTENT']))